In [1]:
import pandas as pd
import requests
import os
import numpy as np
from ratelimit import limits, sleep_and_retry

A notebook to calculate "lineage scores" for every UniRef cluster used to create the ColabFold clusters.

In [2]:
# url = "https://rest.uniprot.org/uniref/stream?format=json&query=%28%28id%3AUniRef100_A0A2G8JTM8%29%29"
# response = requests.get(url).json()
# print(response['results'].keys())
# # df = pd.json_normalize(response['results'])
# # taxon = response['results'][0]['commonTaxon']
# # print(taxon)
# # df = pd.json_normalize(taxon)
# # df.head()
# # tax = df.iloc[0]['taxonId']
# # print(tax)

In [39]:
def get_unirefs(fp):
    '''
    Read the UniRef IDs from the a3m file
    '''

    unirefs = []

    with open(fp) as f:
        for line in f:
            if "UniRef" in line:
                unirefs.append(line.strip())

    # Remove the '>' character from the beginning of each UniRef ID (e.g. '>UniRef100_A0A8B9YR53')
    cleaned_unirefs = [uniref.split(">")[1] for uniref in unirefs]

    return cleaned_unirefs

# @sleep_and_retry
# @limits(calls=1, period=timedelta(seconds=1).total_seconds())
def get_uniref_taxonId(uniref):
    '''
    Get the seed taxonID for a given UniRef cluster
    '''

    # UniProt REST API URL
    url = f"https://rest.uniprot.org/uniref/stream?format=json&query=%28%28id%3A{uniref}%29%29"

    response = requests.get(url).json()
    if len(response['results']) == 0:
        return None
    else:
        taxon = response['results'][0]['commonTaxon']
        df = pd.json_normalize(taxon)
        taxonId = df.iloc[0]['taxonId']

        return taxonId

# @sleep_and_retry
# @limits(calls=1, period=timedelta(seconds=1).total_seconds())
def get_uniprot_taxonId(uniprot):
    '''
    Get the taxonId for a given UniProt ID
    '''

    # UniProt REST API URL
    url = f'https://rest.uniprot.org/uniprotkb/{uniprot}.json'
    response = requests.get(url).json()
    taxonId = response['organism']['taxonId']

    return taxonId

# @sleep_and_retry
# @limits(calls=1, period=timedelta(seconds=1).total_seconds())
def get_lineage(taxonId):
    '''
    Get the lineage for a given taxonID
    '''
    
    url = f"https://rest.uniprot.org/taxonomy/{taxonId}.json"
    response = requests.get(url).json()
    if 'lineage' not in response.keys():
        return [taxonId]
    else:
        lineage = response['lineage']
        df = pd.json_normalize(lineage)

        # Turn taxonId column to list of strings
        taxa_list = df['taxonId'].tolist()

        return taxa_list

def cluster_lineages(df, path):
    '''
    Calculate the 'lineage score' (i.e. the number of shared taxa with the original organism) for each cluster
    '''

    # Keep track of which UniProt ID is being queried to avoid redundant queries
    current_uniprot = ''
    current_lineage = []

    # Empty Dataframe to keep track of the lineage scores
    lineage_df_list = []

    # Iterate over the rows in the DataFrame
    for index, row in df.iterrows():

        uniprot = row['uniprot']
        cluster = row['cluster']

        # If the UniProt ID is different from the last one queried, get the lineage
        if uniprot != current_uniprot:
            current_uniprot = uniprot

            # Get the lineage for the UniProt ID
            uniprot_taxon = get_uniprot_taxonId(uniprot)
            current_lineage = get_lineage(uniprot_taxon)

        # Open the cluster file to get the UniRef IDs
        fn = f'{uniprot}_{cluster}.a3m'
        fp = os.path.join(path, uniprot, fn) # File path looks like data/O08967/O08967_000.a3m
        unirefs = get_unirefs(fp)

        # Iterate over the UniRef IDs to get their lineages
        for uniref in unirefs:
            print(f'UniProt: {uniprot}, Cluster: {cluster}, UniRef: {uniref}')
            cluster_taxon = get_uniref_taxonId(uniref)

            if cluster_taxon == None:
                cluster_lineage = []

                # Create temp_df to append to lineage_df_list
                temp_df = pd.DataFrame([[uniprot, row['state'], row['conformation'], cluster, fn, uniref, 
                                        cluster_taxon, current_lineage, cluster_lineage]],
                                        columns=['uniprot', 'state', 'conformation', 'cluster', 'filename', 'uniref', 
                                                 'taxonId', 'uniprot_lineage', 'cluster_lineage'])
                
                # Append temp_df to lineage_df_list
                lineage_df_list.append(temp_df)

            else:
                print(f'Taxon: {cluster_taxon}')
                cluster_lineage = get_lineage(cluster_taxon)

                # Add values to temp_df
                temp_df = pd.DataFrame([[uniprot, row['state'], row['conformation'], cluster, fn, uniref, 
                                        cluster_taxon, current_lineage, cluster_lineage]],
                                        columns=['uniprot', 'state', 'conformation', 'cluster', 'filename', 'uniref', 
                                                 'taxonId', 'uniprot_lineage', 'cluster_lineage'])

                # Append temp_df to lineage_df_list
                lineage_df_list.append(temp_df)

    # Concatenate the lineage_df_list into a single DataFrame
    lineage_df = pd.concat(lineage_df_list).reset_index(drop=True)

    return lineage_df

def calculate_lineage_scores(df):

    # Take the dataframe we made in cluster_lineages and get our lineage scores
    for idx, row in df.iterrows():
        uniprot_lineage = row['uniprot_lineage']
        cluster_lineage = row['cluster_lineage']

        # Calculate the lineage score
        lineage_score = len(set(uniprot_lineage).intersection(cluster_lineage))

        # Calculate the lineage lengths
        uniprot_lineage_length = len(uniprot_lineage)
        cluster_lineage_length = len(cluster_lineage)

        # Add the lineage score to the DataFrame
        df.loc[idx, 'lineage_score'] = lineage_score
        df.loc[idx, 'uniprot_lineage_length'] = uniprot_lineage_length
        df.loc[idx, 'cluster_lineage_length'] = cluster_lineage_length
    
    return df

In [4]:
df = pd.read_csv('./project_pipeline/data/two-state_af_cf.tsv', sep='\t')

path = './project_pipeline/data/input/Colabfold_cif/autoinhibited'

# Retrieve the lineage scores
lineage_df = cluster_lineages(df, path)

UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_A0A812VSQ5
Taxon: 2952
UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_A0A2G8JTM8
Taxon: 307972
UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_UPI001469F9E4
Taxon: 52239
UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_A0A8C7Z9G0
Taxon: 183150
UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_UPI001E1BC1D8
Taxon: 35525
UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_A0A814HDE6
Taxon: 1234261
UniProt: O08967, Cluster: U10-005, UniRef: UniRef100_A0A7S2NR26
Taxon: 156173
UniProt: O08967, Cluster: 031, UniRef: UniRef100_A0A024UJ15
Taxon: 157072
UniProt: O08967, Cluster: 031, UniRef: UniRef100_W4FH40
Taxon: 112090
UniProt: O08967, Cluster: U10-001, UniRef: UniRef100_A0A553MZ66
Taxon: 623744
UniProt: O08967, Cluster: U10-001, UniRef: UniRef100_UPI000C790C5F
Taxon: 1294199
UniProt: O08967, Cluster: U10-001, UniRef: UniRef100_UPI000623AF91
Taxon: 83485
UniProt: O08967, Cluster: U10-001, UniRef: UniRef100_UPI000C76D0A1


In [38]:
# Get the lineage scores
lineage_scores = calculate_lineage_scores(lineage_df)

lineage_scores.to_csv('./project_pipeline/data/lineage_scores.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
df = pd.read_csv('./project_pipeline/data/lineage_scores.csv')

df.head()

uniprot_lineage = df.loc[1, 'uniprot_lineage']
cluster_lineage = df.loc[1, 'cluster_lineage']

uniprot_lineage = [str(i) for i in uniprot_lineage.strip('[]').split(', ')]
cluster_lineage = [str(i) for i in cluster_lineage.strip('[]').split(', ')]

print(len(set(uniprot_lineage).intersection(cluster_lineage)))

print(uniprot_lineage)
print(cluster_lineage)
df.head()

7
['131567', '2759', '33154', '33208', '6072', '33213', '33511', '7711', '89593', '7742', '7776', '117570', '117571', '8287', '1338369', '32523', '32524', '40674', '32525', '9347', '1437010', '314146', '314147', '9989', '1963758', '337687', '10066', '39107', '10088', '862507']
['131567', '2759', '33154', '33208', '6072', '33213', '33511', '7586', '133551', '7624', '7705', '7681', '7682', '7687', '307971']


,uniprot,state,conformation,cluster,filename,uniref,taxonId,uniprot_lineage,cluster_lineage,lineage_score,uniprot_lineage_length,cluster_lineage_length
0,O08967,Autoinhibited,Closed,U10-005,O08967_U10-005.a3m,UniRef100_A0A812VSQ5,2952.0,"[131567, 2759, 33154, 33208, 6072, 33213, 3351...","[131567, 2759, 2698737, 33630, 2864, 89954, 25...",13,30,32
1,O08967,Autoinhibited,Closed,U10-005,O08967_U10-005.a3m,UniRef100_A0A2G8JTM8,307972.0,"[131567, 2759, 33154, 33208, 6072, 33213, 3351...","[131567, 2759, 33154, 33208, 6072, 33213, 3351...",13,30,32
2,O08967,Autoinhibited,Closed,U10-005,O08967_U10-005.a3m,UniRef100_UPI001469F9E4,52239.0,"[131567, 2759, 33154, 33208, 6072, 33213, 3351...","[131567, 2759, 33154, 33208, 6072, 33213, 3351...",13,30,32
3,O08967,Autoinhibited,Closed,U10-005,O08967_U10-005.a3m,UniRef100_A0A8C7Z9G0,183150.0,"[131567, 2759, 33154, 33208, 6072, 33213, 3351...","[131567, 2759, 33154, 33208, 6072, 33213, 3351...",13,30,32
4,O08967,Autoinhibited,Closed,U10-005,O08967_U10-005.a3m,UniRef100_UPI001E1BC1D8,35525.0,"[131567, 2759, 33154, 33208, 6072, 33213, 3351...","[131567, 2759, 33154, 33208, 6072, 33213, 3331...",13,30,32
